In [1]:
# Load Packages
from igraph import *
import pickle
import matplotlib.pyplot as plt
import random
import glob
import numpy as np

In [0]:
# Generate the networks with varying degrees of clusterings/ transitivity
for i in (40,60,80,100,120,140):
    print(i)
    # Generate networks with a power between [2.2-2.4], ...
    counter = 0
    r2 = 200 + i
    r1 = r2 - 40
    while counter < 100:
        # Generate many networks with a power between 2.0-3.2 and a number of edges between 10 000 and 2 000 000
        no_nodes = 10000
        no_edges = random.sample(range(10000,2000000),1)[0]
        power = random.sample(range(r1,r2),1)[0]/100.
        try:
            G = Graph.Static_Power_Law(n=no_nodes,m=no_edges,exponent=power)
            fit = power_law_fit(G.degree())
            if fit.alpha >= (r1+20)/100 and fit.alpha <= (r2)/100:
                transitivity = G.transitivity_undirected()
                if transitivity < 0.2:
                    # make sure that the assigned power was achieved b/c this may not always be the case
                    # make sure that for all sets the transitivity is always the same [0,0.2]
                    file_name = "./Graphs/Graph_{}_{}.pkl".format(r2/100,counter)
                    pickle.dump(G,open(file_name, 'wb'))
                    counter += 1
        except InternalError:
            print("Invalid no. of edges")

In [0]:
# Get clustering, transitivity and power from all generated networks
clusterings = []
transitivities = []
powers = []
file_names = glob.glob('./Graphs/*.pkl')
for file_name in file_names:
    G = pickle.load(open(file_name,"rb"))
    transitivity = G.transitivity_undirected() #ratio of the triangles and connected triplets in the graph
    average_clustering = G.transitivity_avglocal_undirected() #average clutering coefficient of all nodes in the network
    fit = power_law_fit(G.degree())
    powers.append(fit.alpha)
    transitivities.append(transitivity)
    clusterings.append(average_clustering)

# Scatter plot of clustering versus power
x = powers
y = transitivities
fs = 18
area = [c*300 for c in clusterings]
plt.scatter(x, y, s=area)
plt.ylabel("Transitivity", fontsize=fs)
plt.xlabel("Power", fontsize=fs)
plt.xticks(fontsize=fs)
plt.yticks(fontsize=fs)
plt.savefig('Fig_Transitivity_clustering_power_n{}.pdf'.format(len(file_names)))
plt.show()

In [0]:
import numpy as np
powers = np.array(powers)
transitivities = np.array(transitivities)

In [0]:
# Check that all power ranges have a similiar distribution of transitivities
for i in (40,60,80,100,120):
    #Power between [2.2-2.4], ... [3.6,3.8]
    r2 = (200 + i)/100.
    r1 = r2 - 0.2
    indcs = np.where(np.logical_and(powers> r1, powers< r2))
    m = round(np.mean(transitivities[indcs]),3)
    s = round(np.std(transitivities[indcs]),3)
    print("[{},{}]: no: {}, mean = {}, sd = {}".format(round(r1,2),r2,len(indcs[0]),m,s))

In [2]:
# Get average degree for a network of a certain power range (degree exponent)
pw = "3.2"
average_degrees = []
file_names = glob.glob('./Graphs/*.pkl')
for g_name in file_names:
    if pw in g_name:
        g = pickle.load(open(g_name,"rb"))
        average_degrees.append(np.mean(g.degree()))
print(np.mean(average_degrees))